## 用jupyter notebook运行不知道总是报错，但是pycharm就不会

In [1]:
import os
import pandas as pd
import numpy as np
# from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
import shutil
import time
import gc
import random
import math
import torch
from torch.utils.data import DataLoader, Dataset 
import transformers
from transformers import TrainingArguments, Trainer, DataCollatorForWholeWordMask
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel
from torch import nn
from torch.optim import Adam, SGD, AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from sklearn.preprocessing import MinMaxScaler

os.environ["WANDB_DISABLED"] = "true"



In [2]:

class evalDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.input = df['input'].values.astype(str)
        self.anchor = df['anchor'].values.astype(str)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.input)

    def __getitem__(self, item):
        inputs = self.input[item]
        anchor = self.anchor[item]

        model_inputs = self.tokenizer(inputs, anchor,
                                      max_length=100,
                                      padding='max_length',
                                      truncation=True)

        # return {**model_inputs}
        return torch.as_tensor(model_inputs['input_ids'], dtype=torch.long), \
               torch.as_tensor(model_inputs['token_type_ids'], dtype=torch.long), \
               torch.as_tensor(model_inputs['attention_mask'], dtype=torch.long)


In [3]:
def valid_fn(valid_loader, model, device):
    model.eval()
    preds = []
    labels = []
    # import pdb; pdb.set_trace() 调试用的
    for step, batch in enumerate(valid_loader):
        input_ids, token_type_ids, attention_mask = [i.to(device) for i in batch]
        with torch.no_grad():
            y_preds = model(input_ids, attention_mask, token_type_ids)
        preds.append(y_preds.logits.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions


In [4]:
tonkenizer = AutoTokenizer.from_pretrained('./deberta-v3-small')
df_context = pd.read_csv('../../uspatent/titles.csv')

eval_df = pd.read_csv('./us-patent-phrase-to-phrase-matching/test.csv')
eval_df = eval_df.merge(df_context, how='left', left_on='context', right_on='code')
eval_df = eval_df[['id', 'anchor', 'target', 'context', 'title']]
eval_df.head()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,id,anchor,target,context,title
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,OPTICS
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES
2,36baf228038e314b,lower trunnion,lower locating,B60,VEHICLES IN GENERAL
3,1f37ead645e7f0c8,cap component,upper portion,D06,TREATMENT OF TEXTILES OR THE LIKE; LAUNDERING;...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRIC COMMUNICATION TECHNIQUE


In [5]:
eval_df['input'] = eval_df['target'] + tonkenizer.sep_token + eval_df['title'].apply(str.lower)
eval_df.head()
# 拿到数据

,id,anchor,target,context,title,input
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,OPTICS,inorganic photoconductor drum[SEP]optics
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,altering gas flow[SEP]combustion apparatus; co...
2,36baf228038e314b,lower trunnion,lower locating,B60,VEHICLES IN GENERAL,lower locating[SEP]vehicles in general
3,1f37ead645e7f0c8,cap component,upper portion,D06,TREATMENT OF TEXTILES OR THE LIKE; LAUNDERING;...,upper portion[SEP]treatment of textiles or the...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRIC COMMUNICATION TECHNIQUE,artificial neural network[SEP]electric communi...


In [6]:
 # 拿到数据
va_dataset = evalDataset(eval_df, tonkenizer)
va_dataloader = DataLoader(va_dataset,
                            batch_size=128,
                            shuffle=False,  # inference不要打乱
                            num_workers=2)  # drop_last=False

In [7]:
model = AutoModelForSequenceClassification.from_pretrained('./deberta-v3-small', num_labels=1)
model.load_state_dict(torch.load('./deberta-v3-small.pth')['model'])
model.to('cuda')
# 开始inference
predictions = []
MMscaler = MinMaxScaler()
outputs = valid_fn(va_dataloader, model, 'cuda')
prediction = outputs.reshape(-1)
predictions.append(MMscaler.fit_transform(prediction.reshape(-1, 1)).reshape(-1))
# 截止
predictions = np.clip(predictions, 0, 1)


Some weights of the model checkpoint at ./deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFo

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
import datasets
submission = datasets.Dataset.from_dict({
    'id': eval_df['id'],
    'score': outputs.flatten()
})

submission.to_csv('submission.csv', index=False)